# Skymodel converter for gps
-----
-  prototype version of skymodel converter
-  for gps model
-  use the latest gammapy version

# Spectral modeling

In [1]:

from gammapy.modeling.models import SkyModel
from astropy import units as u
import numpy as np
import os


dict_possibleattributeslist={
  # "@name"  : "name", 
  # "@value" : "value",
  "@error" : "error",  
  "@scale" : "scale", 
  "@min"   : "min",
  "@max"   : "max",
  "@free"  : "frozen"
  }

def get_values(parameters):
  paramvalues={}
  for parameter in parameters :    
    paramvalues[parameter['@name']]=float(parameter['@value'])*float(parameter['@scale'])
  return paramvalues

def get_attributes(parameters):
  params_attributes={}
  for key in dict_possibleattributeslist.keys() :
    params_attributes[key]={}
    for parameter in parameters :
      if key in parameter:
        params_attributes[key][parameter['@name']]=parameter[key]
  return params_attributes

########################################################
################   Spectral Models   ###################
########################################################
# Dictionary for spectral model names
########################################################
# from gammapy.modeling.models import SPECTRAL_MODEL_REGISTRY
# print(SPECTRAL_MODEL_REGISTRY)
dict_spectralmodel={}
#              -- ctool --                                   --  gammapy --
dict_spectralmodel['FileFunction']                     ="TemplateSpectralModel"
dict_spectralmodel['PowerLaw']                         ="PowerLawSpectralModel"
dict_spectralmodel['NodeFunction']                     ="PiecewiseNormSpectralModel * ConstantSpectralModel"
dict_spectralmodel['ExponentialCutoffPowerLaw']        ="ExpCutoffPowerLawSpectralModel"
dict_spectralmodel['ExpCutoff']                        ="ExpCutoffPowerLawSpectralModel"
dict_spectralmodel['SuperExponentialCutoffPowerLaw']   ="SuperExpCutoffPowerLaw3FGLSpectralModel"
dict_spectralmodel['Composite']                        ="summation of multiple models"
dict_spectralmodel['LogParabola']                      ="LogParabolaSpectralModel"
dict_spectralmodel['Constant']                         ="ConstantSpectralModel"
dict_spectralmodel['SmoothBrokenPowerLaw']             ="SmoothBrokenPowerLawSpectralModel"
dict_spectralmodel['BrokenPowerLaw']                   ="BrokenPowerLawSpectralModel"
dict_spectralmodel['Multiplicative']                   ="CompoundSpectralModel"
dict_spectralmodel['Exponential']                      ="ExpCutoffPowerLawNormSpectralModel"

###########################################
# Spectral model conversion functions
###########################################
from gammapy.modeling.models import (
  PowerLawSpectralModel,
  PiecewiseNormSpectralModel,
  ExpCutoffPowerLawSpectralModel,
  SuperExpCutoffPowerLaw3FGLSpectralModel,
  ConstantSpectralModel,
  TemplateSpectralModel,
  LogParabolaSpectralModel,
  SmoothBrokenPowerLawSpectralModel,
  BrokenPowerLawSpectralModel,
  CompoundSpectralModel,
  ExpCutoffPowerLawNormSpectralModel
)
def generate_spectralmodel(ct_spectralinfo):
  spectraltype=ct_spectralinfo["@type"]
  # print('__________  paraeters in ctools format _______________________________')
  # print("Spectral type: ", ct_spectralinfo["@type"], "with ", len(ct_spectral_parameters), "parameters")
  # print("Spectral type: ", ct_spectralinfo["@type"], "with ", len(ct_spectral_parameters), "parameters", ".... and file: ",ct_spectralinfo["file"])
  # print(ct_spectral_parameters)    
  # print('__________________________________________________________________________')    
  print("@ generate_spectralmodel: This spectral model is ", spectraltype)
#             spectraltype                                  function name
#              -- ctool --                                   --  gammapy --
  if spectraltype=="FileFunction":
    parameters = ct_spectralinfo["parameter"]
    filename=ct_spectralinfo["@file"]
    gp_spectralmodel                          =set_TemplateSpectralModel(parameters,filename)  
  if spectraltype=="PowerLaw":
    parameters = ct_spectralinfo["parameter"]
    gp_spectralmodel                          =set_PowerLawSpectralModel(parameters)
  if spectraltype=="NodeFunction":
    nodes= ct_spectralinfo["node"]
    gp_spectralmodel                          =set_PiecewiseNormSpectralModel(nodes)
  if spectraltype=="ExponentialCutoffPowerLaw":
    parameters = ct_spectralinfo["parameter"]
    gp_spectralmodel                          =set_ExpCutoffPowerLawSpectralModel_1(parameters)
  if spectraltype=="ExpCutoff":
    parameters = ct_spectralinfo["parameter"]
    gp_spectralmodel                          =set_ExpCutoffPowerLawSpectralModel_2(parameters)
  if spectraltype=="SuperExponentialCutoffPowerLaw" :
    parameters = ct_spectralinfo["parameter"]
    gp_spectralmodel                          =set_SuperExpCutoffPowerLaw3FGLSpectralModel(parameters)
  if spectraltype=="Composite" :
    gp_spectralmodel                          =set_sum_of_multiple_models(ct_spectralinfo)
  if spectraltype=="LogParabola":
    parameters = ct_spectralinfo["parameter"]
    gp_spectralmodel                          =set_LogParabolaSpectralModel(parameters)
  if spectraltype=="Constant":
    parameters = ct_spectralinfo["parameter"]
    gp_spectralmodel                          =set_ConstantSpectralModel(parameters)
  if spectraltype=="SmoothBrokenPowerLaw":
    parameters = ct_spectralinfo["parameter"]
    gp_spectralmodel                          =set_SmoothBrokenPowerLawSpectralModel(parameters)
  if spectraltype=="BrokenPowerLaw":
    parameters = ct_spectralinfo["parameter"]
    gp_spectralmodel                          =set_BrokenPowerLawSpectralModel(parameters)
  if spectraltype=="Multiplicative":
    gp_spectralmodel                          =set_CompoundSpectralModel(ct_spectralinfo)
                   
  # gp_spectralmodel                          =set_DefaultSpectalModel(parameters)
  return gp_spectralmodel

###########################################
#   _DefaultSpectarlModel
###########################################
def set_DefaultSpectalModel(parameters) :
  spectralmodel = ExpCutoffPowerLawSpectralModel(
      # amplitude=3.75999995028131e-17 * u.Unit("cm−2 s−1 MeV−1"),
      # index=-2.39000010490417*(-1),
      # lambda_=1./(14300000.1907349 * u.Unit("MeV")),
      # reference=1 *1000000* u.Unit("MeV"),
  )
  return spectralmodel

# https://docs.gammapy.org/0.19/tutorials/api/models.html#Spectral-Models

###########################################
#   FileFunction  to  TemplateSpectralModel
########################################### 
# http://cta.irap.omp.eu/ctools/users/user_manual/models_spectral.html#file-function
# NOTE: only one parameter
#-> 
# https://docs.gammapy.org/0.19/api/gammapy.modeling.models.TemplateSpectralModel.html#gammapy.modeling.models.TemplateSpectralModel 
# https://docs.gammapy.org/0.19/modeling/gallery/spectral/plot_template_spectral.html#template-spectral-model
# Mspectral(E)=N0dNdE∣∣∣file
# where
# N0 = Normalization
def set_TemplateSpectralModel(parameters, filename) :
  # print("----------->",filename)
  print(os.path.join(modelfilepath, filename))
  # inputfilepath=os.path.join(modelfilepath, filename)
  # import pandas as pd
  # data = pd.read_csv(inputfilepath)
  # replace StringIO(x) with open('file.csv', 'r')
      # idx = next(idx for idx, row in enumerate(reader) if len(row) > 0)  # 4
  # print('skip first {nrows} rows in the file to load========='.format(nrows = idx))
  data = np.loadtxt(os.path.join(modelfilepath, filename),skiprows=1)

  
  paramvalues=get_values([parameters])
  energies=data[:,0]*u.MeV
  fluxes=data[:,1]*paramvalues['Normalization']*u.Unit("MeV-1 s-1 cm-2")
  spectralmodel = TemplateSpectralModel(
    energy=energies, values=fluxes
    )
  return spectralmodel

###########################################
#   PowerLaw  to  PowerLawSpectralModel
########################################### 
# http://cta.irap.omp.eu/ctools/users/user_manual/models_spectral.html#power-law
# -> 
# https://docs.gammapy.org/0.19/modeling/gallery/spectral/plot_powerlaw.html#sphx-glr-modeling-gallery-spectral-plot-powerlaw-py
# https://docs.gammapy.org/0.19/api/gammapy.modeling.models.PowerLawSpectralModel.html#gammapy.modeling.models.PowerLawSpectralModel.amplitude
# - ctools definition -                                        -- gammapy --  
# Mspectral(E)=k0(E/E0)^γ   
# where                                  
# k0  = Prefactor (phcm−2s−1MeV−1)                       = amplitude   TeV-1 cm-2 s-1           
# γ = Index                                              = index
# E0 = PivotEnergy (MeV)                                 = reference   * u.TeV
def set_PowerLawSpectralModel(parameters):
  paramvalues=get_values(parameters)
  get_attributes(parameters)  
  spectralmodel = PowerLawSpectralModel(
    amplitude = paramvalues['Prefactor']* u.Unit("cm-2 s-1 MeV-1"),
    index     = paramvalues['Index']*(-1),
    refecence = paramvalues['PivotEnergy']*u.Unit("MeV")
  )
  params_attributes=get_attributes(parameters)
  return spectralmodel

# - ctools alternative definition -  -> not used in the gcs xml    
# Mspectral(E)=N(γ+1)Eγ / (E_{γ+1max}−E_{γ+1min})
# N = PhotonFlux (phcm−2s−1)
# γ = Index
# Emin = LowerLimit (MeV)
# Emax = UpperLimit (MeV)

###########################################
#   NodeFunction  to  PiecewiseNormSpectralModel * ConstantSpectralModel
########################################### 
# http://cta.irap.omp.eu/ctools/users/user_manual/models_spectral.html#node-function
# - ctools definition -                    
# This spectral model component implements a generalised broken power law
#  which is defined by a set of energy and intensity values (the so called nodes) 
#  that are piecewise connected by power laws. Energies are given in units of MeV, 
#  intensities are given in units of phcm−2s−1MeV−1.
# -> 
# -- gammapy --  
# --- PiecewiseNormSpectralModel --- 
# https://docs.gammapy.org/0.19/modeling/gallery/spectral/plot_piecewise_norm_spectral.html#piecewise-norm-spectral
# https://docs.gammapy.org/0.19/api/gammapy.modeling.models.PiecewiseNormSpectralModel.html#gammapy.modeling.models.PiecewiseNormSpectralModel
# --- ConstantSpectralModel --- 
# https://docs.gammapy.org/0.19/modeling/gallery/spectral/plot_constant_spectral.html#constant-spectral-model
# https://docs.gammapy.org/0.19/api/gammapy.modeling.models.ConstantSpectralModel.html#gammapy.modeling.models.ConstantSpectralModel
# 
#  
def set_PiecewiseNormSpectralModel(nodes) :
  energies=[]
  fluxes=[]
  for node in nodes:
    parameters = node["parameter"]
    paramvalues= get_values(parameters)
    energies=np.append(energies,paramvalues['Energy'])
    fluxes=np.append(fluxes,paramvalues['Intensity'])
  energies=energies*u.Unit("MeV")
  fluxes=fluxes#*u.Unit("MeV-1 s-1 cm-2")
  spectralmodel= PiecewiseNormSpectralModel(
    energy=energies, norms=fluxes
  )
  spectralmodel = spectralmodel * ConstantSpectralModel(const="1 / (cm2 s MeV)")
  return spectralmodel

###########################################
#   ExponentialCutoffPowerLaw to ExpCutoffPowerLawSpectralModel
###########################################
# http://cta.irap.omp.eu/ctools/users/user_manual/models_spectral.html#exponentially-cut-off-power-law
# -> 
# https://docs.gammapy.org/0.19/modeling/gallery/spectral/plot_exp_cutoff_powerlaw.html#sphx-glr-modeling-gallery-spectral-plot-exp-cutoff-powerlaw-py
# https://docs.gammapy.org/0.19/api/gammapy.modeling.models.ExpCutoffPowerLawSpectralModel.html#gammapy.modeling.models.ExpCutoffPowerLawSpectralModel
# - ctools definition -                                        -- gammapy --  
# Mspectral(E)=k0(E/E0)^γexp(−E/Ecut)                𝜙(𝐸)=𝜙0⋅(𝐸/𝐸0)^−Γexp(−(𝜆𝐸)^𝛼)              
# where                                                 
# k0 = Prefactor (phcm−2s−1MeV−1)                       = amplitude
# γ = Index                                             = index
# E0 = PivotEnergy (MeV)                                = reference
# Ecut = CutoffEnergy (MeV)                             = 1/ lambda_
def set_ExpCutoffPowerLawSpectralModel_1(parameters) : 
  # print(parameters)
  paramvalues=get_values(parameters)
  get_attributes(parameters)  
  spectralmodel = ExpCutoffPowerLawSpectralModel(
    amplitude = paramvalues['Prefactor']* u.Unit("cm-2 s-1 MeV-1"),
    index     = paramvalues['Index']*(-1),
    refecence = paramvalues['PivotEnergy']*u.Unit("MeV"),
    lambda_   = 1./(paramvalues['CutoffEnergy']*u.Unit("MeV"))  
  )
  params_attributes=get_attributes(parameters)
  if 'Index' in params_attributes['@min']:
    spectralmodel.index.min=float(params_attributes['@min']['Index'])*(-1)*float(params_attributes['@scale']['Index'])
  if 'Index' in params_attributes['@max']:
    spectralmodel.index.max=float(params_attributes['@max']['Index'])*(-1)*float(params_attributes['@scale']['Index'])

  return spectralmodel
###########################################
#   ExpCutoff to ExpCutoffPowerLawSpectralModel
###########################################
# http://cta.irap.omp.eu/ctools/users/user_manual/models_spectral.html#exponentially-cut-off-power-law
# -> 
# https://docs.gammapy.org/0.19/modeling/gallery/spectral/plot_exp_cutoff_powerlaw.html#sphx-glr-modeling-gallery-spectral-plot-exp-cutoff-powerlaw-py
# https://docs.gammapy.org/0.19/api/gammapy.modeling.models.ExpCutoffPowerLawSpectralModel.html#gammapy.modeling.models.ExpCutoffPowerLawSpectralModel
# - ctools definition -                                        -- gammapy --  
# Mspectral(E)=k0(E/E0)^γexp(−E/Ecut)                𝜙(𝐸)=𝜙0⋅(𝐸/𝐸0)^−Γexp(−(𝜆𝐸)^𝛼)              
# where                                                 
# k0 = Prefactor (phcm−2s−1MeV−1)                       = amplitude
# γ = Index                                             = index
# E0 = PivotEnergy (MeV)                                = reference
# Ecut = CutoffEnergy (MeV)                             = 1/ lambda_
# For compatibility with the Fermi/LAT ScienceTools
#  the model type ExponentialCutoffPowerLaw can be replaced by ExpCutoff
#  and the parameters CutoffEnergy by Cutoff and PivotEnergy by Scale.
def set_ExpCutoffPowerLawSpectralModel_2(parameters) : 
  # print(parameters)
  paramvalues=get_values(parameters)
  get_attributes(parameters)  
  spectralmodel = ExpCutoffPowerLawSpectralModel(
    amplitude = paramvalues['Prefactor']* u.Unit("cm-2 s-1 MeV-1"),
    index     = paramvalues['Index']*(-1),
    refecence = paramvalues['Scale']*u.Unit("MeV"),
    lambda_   = 1./(paramvalues['Cutoff']*u.Unit("MeV"))  
  )
  params_attributes=get_attributes(parameters)
  if 'Index' in params_attributes['@min']:
    spectralmodel.index.min=float(params_attributes['@min']['Index'])*(-1)*float(params_attributes['@scale']['Index'])
  if 'Index' in params_attributes['@max']:
    spectralmodel.index.max=float(params_attributes['@max']['Index'])*(-1)*float(params_attributes['@scale']['Index'])

  return spectralmodel

###########################################
#   SuperExponentialCutoffPowerLaw  to  SuperExpCutoffPowerLaw3FGLSpectralModel
###########################################     
# http://cta.irap.omp.eu/ctools/users/user_manual/models_spectral.html#super-exponentially-cut-off-power-law 
# -> 
# https://docs.gammapy.org/0.19/modeling/gallery/spectral/plot_super_exp_cutoff_powerlaw_3fgl.html#sphx-glr-modeling-gallery-spectral-plot-super-exp-cutoff-powerlaw-3fgl-py
# https://docs.gammapy.org/0.19/api/gammapy.modeling.models.SuperExpCutoffPowerLaw3FGLSpectralModel.html#gammapy.modeling.models.SuperExpCutoffPowerLaw3FGLSpectralModel
# - ctools definition -                                        -- gammapy --  
# Mspectral(E)=k0(EE0)γexp(−(EEcut)α)                    𝜙(𝐸)=𝜙0⋅(𝐸/𝐸0)−Γ1exp((𝐸0/𝐸𝐶)Γ2−(𝐸/𝐸𝐶)Γ2)     
# where                                             where                                     
# k0 = Prefactor (phcm−2s−1MeV−1)                    𝜙0 = amplitude
# γ = Index1                                         Γ1 = index_1 
# α = Index2                                         Γ2 = index_2
# E0 = PivotEnergy (MeV)                             𝐸0 = reference             
# Ecut = CutoffEnergy (MeV)                          𝐸𝐶 = ecut
#
def set_SuperExpCutoffPowerLaw3FGLSpectralModel(parameters) : 
  paramvalues=get_values(parameters)
  get_attributes(parameters)  
  spectralmodel = SuperExpCutoffPowerLaw3FGLSpectralModel(
    amplitude = paramvalues['Prefactor']* u.Unit("cm-2 s-1 MeV-1") * np.exp(-(paramvalues['PivotEnergy']/paramvalues['CutoffEnergy'])**paramvalues['Index2']),
    index_1 = paramvalues['Index1']*(-1),
    index_2 = paramvalues['Index2'],
    reference = paramvalues['PivotEnergy']*u.Unit("MeV"),
    ecut = paramvalues['CutoffEnergy']*u.Unit("MeV")
  )

  return spectralmodel

###########################################
#   Composite  to  summation of multiple models
########################################### 
# http://cta.irap.omp.eu/ctools/users/user_manual/models_spectral.html#composite-model
#->
# https://docs.gammapy.org/0.19/tutorials/api/models.html#Implementing-a-custom-model
# - ctools definition -                                      
# Mspectral(E)=∑i=0N−1M(i)spectral(E)
#where 
# M(i)spectral(E) is any spectral model component
#  (including another composite model),
#  and N is the number of model components
#  that are combined.
# As far as in the gps data, 
# the models included are only LogParabola and SuperExponentialCutoffPowerLaw
#   -- gammapy --  
#  

def set_sum_of_multiple_models(ct_spectralinfo) :
  ct_spectralmodels=ct_spectralinfo["spectrum"]
  spectralmodel=None
  for ct_spectralmodel in ct_spectralmodels:
    if spectralmodel==None : # the first model
      spectralmodel=generate_spectralmodel(ct_spectralmodel)
    else : # from second on
      spectralmodel_to_add=generate_spectralmodel(ct_spectralmodel)
      spectralmodel_to_add.amplitude.frozen = True
      spectralmodel=spectralmodel+spectralmodel_to_add

  return spectralmodel

###########################################
#   LogParabola  to  LogParabolaSpectralModel
########################################### 
# http://cta.irap.omp.eu/ctools/users/user_manual/models_spectral.html#log-parabola
# -> 
# https://docs.gammapy.org/0.19/modeling/gallery/spectral/plot_logparabola.html#sphx-glr-modeling-gallery-spectral-plot-logparabola-py
# https://docs.gammapy.org/0.19/api/gammapy.modeling.models.LogParabolaSpectralModel.html#gammapy.modeling.models.LogParabolaSpectralModel
# - ctools definition -                                     -- gammapy --  
# Mspectral(E)=k0(E/E0)^(γ+ηln(E/E0))                 𝜙(𝐸)=𝜙0(𝐸/𝐸0)^(−𝛼−𝛽log(𝐸/𝐸0))
# where                                                              
# k0 = Prefactor (phcm−2s−1MeV−1)                      𝜙0 = amplitude
# γ = Index                                            𝐸0 = reference
# η = Curvature                                        𝛼  =  alpha
# E0 = PivotEnergy (MeV)                               𝛽  =  beta

def set_LogParabolaSpectralModel(parameters):
  paramvalues=get_values(parameters)
  get_attributes(parameters)  
  spectralmodel = LogParabolaSpectralModel(
    amplitude = paramvalues['Prefactor']* u.Unit("cm-2 s-1 MeV-1"),
    alpha = paramvalues['Index']*(-1),
    beta = paramvalues['Curvature'],
    reference = paramvalues['PivotEnergy']*u.Unit("MeV"),
  )
  return spectralmodel

###########################################
#   Constant  to  ConstantSpectralModel
########################################### 
# http://cta.irap.omp.eu/ctools/users/user_manual/models_spectral.html
# NOTE: only one parameter
# -> 
# https://docs.gammapy.org/0.19/modeling/gallery/spectral/plot_constant_spectral.html
# https://docs.gammapy.org/0.19/api/gammapy.modeling.models.ConstantSpectralModel.html#gammapy.modeling.models.ConstantSpectralModel
# - ctools definition -                                     -- gammapy --  
# Mspectral(E)=N0
# where
# N0 = Normalization (phcm−2s−1MeV−1)                   = const   TeV-1 cm-2 s-1      
def set_ConstantSpectralModel(parameters):
  # print(parameters)
  # NOTE: Only one parameter, thus parameters casted as []
  paramvalues=get_values([parameters])    
  get_attributes([parameters])  
  spectralmodel = ConstantSpectralModel(
    const = paramvalues['Normalization']* u.Unit("cm-2 s-1 MeV-1")
  )
  return spectralmodel



###########################################
#   SmoothBrokenPowerLaw  to  SmoothBrokenPowerLawSpectralModel
########################################### 
# http://cta.irap.omp.eu/ctools/users/user_manual/models_spectral.html#smoothly-broken-power-law
#->
# https://docs.gammapy.org/0.19/modeling/gallery/spectral/plot_smooth_broken_powerlaw.html#smooth-broken-powerlaw-spectral-model
# https://docs.gammapy.org/0.19/api/gammapy.modeling.models.SmoothBrokenPowerLawSpectralModel.html#gammapy.modeling.models.SmoothBrokenPowerLawSpectralModel
# - ctools definition -                                     -- gammapy --  
# Mspectral(E)=k0(E/E0)^γ1[1+(E/Eb)^{(γ1−γ2)/β}]^−β     𝜙(𝐸)=𝜙0⋅(𝐸/𝐸0)^−Γ1[1+(𝐸/𝐸𝑏𝑟𝑒𝑎𝑘)^{(Γ2−Γ1)/𝛽}]^−𝛽
# where                                                                                             
# k0 = Prefactor (phcm−2s−1MeV−1)                            Γ1 = index1    
# γ1 = Index1                                                Γ2 = index2
# E0 = PivotEnergy                                           𝜙0 = amplitude
# γ2 = Index2                                                𝐸0 =reference
# Eb = BreakEnergy (MeV)                                     𝐸𝑏𝑟𝑒𝑎𝑘 = ebreak
# β = BreakSmoothness                                        𝛽 = beta

def set_SmoothBrokenPowerLawSpectralModel(parameters): 
  paramvalues=get_values(parameters)    
  get_attributes(parameters)  
  spectralmodel = SmoothBrokenPowerLawSpectralModel(
    amplitude = paramvalues['Prefactor']* u.Unit("cm-2 s-1 MeV-1"),
    index1    = paramvalues['Index1']*(-1),
    index2    = paramvalues['Index2']*(-1),
    reference = paramvalues['PivotEnergy']*u.Unit("MeV"), 
    ebreak    = paramvalues['BreakEnergy']*u.Unit("MeV"),
    beta      = paramvalues['BreakSmoothness']
  )
  return spectralmodel  
###########################################
#   BrokenPowerLaw  to  BrokenPowerLawSpectralModel
########################################### 
# http://cta.irap.omp.eu/ctools/users/user_manual/models_spectral.html#broken-power-law
#->
# https://docs.gammapy.org/0.19/modeling/gallery/spectral/plot_broken_powerlaw.html#sphx-glr-modeling-gallery-spectral-plot-broken-powerlaw-py
# https://docs.gammapy.org/0.19/api/gammapy.modeling.models.BrokenPowerLawSpectralModel.html#gammapy.modeling.models.BrokenPowerLawSpectralModel
# - ctools definition -                                     -- gammapy --  
# Mspectral(E)=k0×                                𝜙(𝐸)=𝑝ℎ𝑖0⋅
#   ⎧ (E/Eb)^γ1  ifE<Eb                               ⎧(𝐸/𝐸0)^−Γ1  ifE<Ebreak
#   ⎩ (E/Eb)^γ2 otherwise                             ⎩(𝐸/𝐸0)^−Γ2  otherwise   
# where
# k0 = Prefactor (phcm−2s−1MeV−1)                   Γ1 = index1
# γ1 = Index1                                       Γ2 = index2  
# γ2 = Index2                                       # 𝜙0 = amplitude  
# Eb = BreakEnergy (MeV)                            𝐸𝑏𝑟𝑒𝑎𝑘 = ebreak             

def set_BrokenPowerLawSpectralModel(parameters): 
  paramvalues=get_values(parameters)
  get_attributes(parameters)  
  spectralmodel = BrokenPowerLawSpectralModel(
    amplitude = paramvalues['Prefactor']* u.Unit("cm-2 s-1 MeV-1"),
    index1    = paramvalues['Index1']*(-1),
    index2    = paramvalues['Index2']*(-1),
    ebreak    = paramvalues['BreakEnergy']*u.Unit("MeV"),
  )
  return spectralmodel  

###########################################
#   Multiplicative  to CompoundSpectralModel
########################################### 
# http://cta.irap.omp.eu/ctools/users/user_manual/models_spectral.html#multiplicative-model
# - ctools definition -               
# 𝑀spectral(𝐸)=∏𝑖=0𝑁−1𝑀(𝑖)spectral(𝐸)
#where 
# 𝑀(𝑖)spectral(𝐸) is any spectral model component 
#  (including another composite model),
#  and 𝑁 is the number of model components that are multiplied.                       
#   -- gammapy --  
#  https://docs.gammapy.org/dev/user-guide/model-gallery/spectral/plot_absorbed.html?highlight=ebl_dominguez11%20fits%20gz#ebl-absorbption-spectral-model

def set_CompoundSpectralModel(ct_spectralinfo) :
  ct_spectralmodels=ct_spectralinfo["spectrum"]
  spectralmodel=None
  for ct_spectralmodel in ct_spectralmodels:
    if spectralmodel==None :
      spectralmodel=generate_spectralmodel(ct_spectralmodel)
      print('first spectrum done')
    else :
      if ct_spectralmodel['@type']=="ExponentialCutoffPowerLaw" : 
        parameters = ct_spectralmodel["parameter"]
        spectralmodel_to_multiply=set_ExpCutoffPowerLawNormSpectralModel(parameters)
        spectralmodel_to_multiply.norm.frozen = True        
        spectralmodel=spectralmodel*spectralmodel_to_multiply
        print('second spectrum done expct')
      if ct_spectralmodel['@type']=="Exponential" :
        innerspectralinfo = ct_spectralmodel["spectrum"]
        innerspectraltype = innerspectralinfo["@type"]
        if innerspectraltype =="PowerLaw" : 
          parameters = innerspectralinfo["parameter"]
          spectralmodel_to_multiply=set_ExpCutoffPowerLawNormSpectralModel(parameters)
          spectralmodel_to_multiply.norm.frozen = True
          spectralmodel=spectralmodel*spectralmodel_to_multiply
          
        print('second spectrum done exp')
  return spectralmodel #* ConstantSpectralModel(const=1*u.Unit("cm2 s MeV"))


###########################################
#   Exponential  to  ExpCutoffPowerLawNormSpectralModel
########################################### 
# http://cta.irap.omp.eu/ctools/users/user_manual/models_spectral.html#exponential-model
# - ctools definition -               
# 𝑀spectral(𝐸)=exp(𝛼𝑀spectral(𝐸))
# where
# 𝑀spectral(𝐸) is any spectral model component
# 𝛼 = Normalization
# -> 
# https://docs.gammapy.org/0.19/modeling/gallery/spectral/plot_exp_cutoff_powerlaw.html#sphx-glr-modeling-gallery-spectral-plot-exp-cutoff-powerlaw-py
# https://docs.gammapy.org/0.19/api/gammapy.modeling.models.ExpCutoffPowerLawSpectralModel.html#gammapy.modeling.models.ExpCutoffPowerLawSpectralModel
def set_ExpCutoffPowerLawNormSpectralModel(parameters) : 
  # print(parameters)
  paramvalues=get_values(parameters)
  # get_attributes(parameters)  
  spectralmodel = ExpCutoffPowerLawNormSpectralModel(
    # amplitude = 1, #* u.Unit("cm-2 s-1 MeV-1"), #paramvalues['Prefactor']* u.Unit("cm-2 s-1 MeV-1"),
    index     = 0, 
    refecence = 1,
    lambda_   = 1./(paramvalues['PivotEnergy']*u.Unit("MeV")),
    alpha = paramvalues['Index']*(-1),
  )
  params_attributes=get_attributes(parameters)
  if 'Index' in params_attributes['@min']:
    spectralmodel.index.min=float(params_attributes['@min']['Index'])*(-1)*float(params_attributes['@scale']['Index'])
  if 'Index' in params_attributes['@max']:
    spectralmodel.index.max=float(params_attributes['@max']['Index'])*(-1)*float(params_attributes['@scale']['Index'])

  return spectralmodel

# spatial modeling

In [2]:
########################################################
################   Spatial Models   ###################
########################################################
# Dictionary for spatial model names
########################################################
from gammapy.modeling.models import SPATIAL_MODEL_REGISTRY
print(SPATIAL_MODEL_REGISTRY)
dict_spatialmodel={}
#              -- ctool --                                   --  gammapy --
dict_spatialmodel['RadialDisk']           ="DiskSpatialModel"#1106
dict_spatialmodel['PointSource']          ="PointSpatialModel"# 260
dict_spatialmodel['RadialShell']          ="ShellSpatialModel"# 159
dict_spatialmodel['SkyDirFunction']       ="PointSpatialModel"#  38
dict_spatialmodel['RadialGaussian']       ="GaussianSpatialModel"#  37
dict_spatialmodel['EllipticalGaussian']   ="GaussianSpatialModel"#  17
dict_spatialmodel['DiffuseMap']           ="TemplateSpatialModel"#  17
dict_spatialmodel['DiffuseMapCube']       ="TemplateSpatialModel"#   3
###########################################
# Spatial model conversion functions
##########################################
from gammapy.maps import Map
from gammapy.modeling.models import (
  DiskSpatialModel,
  PointSpatialModel,
  ShellSpatialModel,
  TemplateSpatialModel,
  GaussianSpatialModel
)
def generate_spatialmodel(ct_spatialinfo):
  spatialtype=ct_spatialinfo["@type"]
  gp_spatialmodel=None
  if spatialtype=="RadialDisk":
    parameters = ct_spatialinfo["parameter"]
    gp_spatialmodel                          =set_DiskSpatialModel(parameters)
  if spatialtype=="PointSource":         
    parameters = ct_spatialinfo["parameter"]
    gp_spatialmodel                          =set_PointSpatialModel(parameters)
  if spatialtype=="RadialShell":    
    parameters = ct_spatialinfo["parameter"]
    gp_spatialmodel                          =set_ShellSpatialModel(parameters)
  if spatialtype=="SkyDirFunction":         
    parameters = ct_spatialinfo["parameter"]
    gp_spatialmodel                          =set_PointSpatialModel(parameters)
  if spatialtype=="RadialGaussian":
    parameters = ct_spatialinfo["parameter"]
    gp_spatialmodel                          =set_GaussianSpatialModel1(parameters)      
  if spatialtype=="EllipticalGaussian":  
    parameters = ct_spatialinfo["parameter"]
    gp_spatialmodel                          =set_GaussianSpatialModel2(parameters)      
  if spatialtype=="DiffuseMap":
    filename=ct_spatialinfo["@file"]
    parameters = ct_spatialinfo["parameter"]
    gp_spatialmodel                          =set_TemplateSpatialModel1(parameters,filename)       
  if spatialtype=="DiffuseMapCube":
    filename=ct_spatialinfo["@file"]
    parameters = ct_spatialinfo["parameter"]
    gp_spatialmodel                          =set_TemplateSpatialModel2(parameters,filename) 
    

# if spatialtype=="DiffuseMapCube":
  return gp_spatialmodel
###########################################
#   RadialDisk  to  DiskSpatialModel
########################################### 
#              -- ctool --         
# http://cta.irap.omp.eu/ctools/users/user_manual/models_spatial.html#radialdisk
# The RadialDisk model describes a uniform intensity distribution within a given radius
# RA is the Right Ascension of the disk centre (degrees)
# DEC is the Declination of the disk centre (degrees)
# Radius is the disk radius (degrees)
# -> 
# https://docs.gammapy.org/0.19/modeling/gallery/spatial/plot_disk.html#sphx-glr-modeling-gallery-spatial-plot-disk-py
# https://docs.gammapy.org/0.19/_modules/gammapy/modeling/models/spatial.html#DiskSpatialModel
# - lon_0, lat_0
# Center position
# - r_0
# 𝑎: length of the major semiaxis, in angular units.
# - e
# Eccentricity of the ellipse (0<𝑒<1).
# - phi
# Rotation angle 𝜙: of the major semiaxis. Increases counter-clockwise from the North direction.
# - edge_width
# Width of the edge. The width is defined as the range within which the smooth edge of the model drops from 95% to 5% of its amplitude. It is given as fraction of r_0.
# - frame{“icrs”, “galactic”}
# Center position coordinate frame
def set_DiskSpatialModel(parameters):
  paramvalues=get_values(parameters)
  get_attributes(parameters)  
  spatialmodel= DiskSpatialModel(
    lon_0=paramvalues['RA']* u.deg, 
    lat_0=paramvalues['DEC']* u.deg,
    r_0=paramvalues['Radius']* u.deg, 
    frame="icrs"
  )
  return spatialmodel
###########################################
#   PointSource     to  PointSpatialModel
###########################################
#   SkyDirFunction  to  PointSpatialModel
########################################### 
#              -- ctool --    
# http://cta.irap.omp.eu/ctools/users/user_manual/models_spatial.html#point-source
# Note
# For compatibility with the Fermi/LAT ScienceTools
#  the model type PointSource can be replaced by SkyDirFunction.
# Note
# There was no source model specified with "GLON" and "GLAT". 
# -> 
#              -- gammapy --    
# https://docs.gammapy.org/0.19/modeling/gallery/spatial/plot_point.html#sphx-glr-modeling-gallery-spatial-plot-point-py
# https://docs.gammapy.org/0.19/api/gammapy.modeling.models.PointSpatialModel.html#gammapy.modeling.models.PointSpatialModel
def set_PointSpatialModel(parameters):
  paramvalues=get_values(parameters)
  get_attributes(parameters)  
  spatialmodel= PointSpatialModel(
    lon_0=paramvalues['RA']* u.deg, 
    lat_0=paramvalues['DEC']* u.deg,
    frame="icrs"
  )
  return spatialmodel
###########################################
#   RadialShell  to  ShellSpatialModel
########################################### 
#              -- ctool --    
# http://cta.irap.omp.eu/ctools/users/user_manual/models_spatial.html#radialshell
# Mspatial(θ)=n0
# ⎧ √‾θout^2−θ^2‾‾− √‾‾θin^2−θ^2‾‾     if θ≤θin
# ⎨ √‾θout^2−θ^2‾‾                     if θin<θ≤θout
# ⎩  0                                 if θ>θout
# where
# RA is the Right Ascension of the shell centre (degrees)
# DEC is the Declination of the shell centre (degrees)
# θout = Radius + Width (degrees)
# θin = Radius (degrees)
# 
# -> 
#              -- gammapy --    
# https://docs.gammapy.org/0.19/modeling/gallery/spatial/plot_shell.html#sphx-glr-modeling-gallery-spatial-plot-shell-py
# https://docs.gammapy.org/0.19/api/gammapy.modeling.models.ShellSpatialModel.html#gammapy.modeling.models.ShellSpatialModel
# 𝜙(𝑙𝑜𝑛,𝑙𝑎𝑡)=3/ {2𝜋(𝑟3𝑜𝑢𝑡−𝑟3𝑖𝑛)}⋅
# ⎧ √‾𝑟𝑜𝑢𝑡^2−𝜃^2‾‾ − √‾ 𝑟𝑖𝑛^2−𝜃^2‾‾  for 𝜃<𝑟𝑖𝑛
# ⎨ √‾𝑟𝑜𝑢𝑡^2−𝜃^2‾‾                  for 𝑟𝑖𝑛≤𝜃<𝑟𝑜𝑢𝑡
# ⎩  0                             for 𝜃>𝑟𝑜𝑢𝑡
#  where 𝜃 is the sky separation and 𝑟out=𝑟in + width
# lon_0, lat_0  = Center position
# radius = Inner radius, 𝑟𝑖𝑛
# width = Shell width
# frame{“icrs”, “galactic”} = Center position coordinate frame
def set_ShellSpatialModel(parameters):
  paramvalues=get_values(parameters)
  get_attributes(parameters)  
  spatialmodel= ShellSpatialModel(
    lon_0=paramvalues['RA']* u.deg, 
    lat_0=paramvalues['DEC']* u.deg,
    radius=paramvalues['Radius']* u.deg,
    width=paramvalues['Width']* u.deg,
    frame="icrs"
  )
  return spatialmodel
###########################################
#   RadialGaussian  to  GaussianSpatialModel
########################################### 
#              -- ctool --    
# http://cta.irap.omp.eu/ctools/users/user_manual/models_spatial.html#radialgaussian
# Mspatial(θ)=1/(2πσ)^2 exp(−1/2 θ^2/σ^2),
# where
# RA is the Right Ascension of the Gaussian centre (degrees)
# DEC is the Declination of the Gaussian centre (degrees)
# σ = Sigma (degrees)
# -> 
#              -- gammapy --    
# https://docs.gammapy.org/0.19/modeling/gallery/spatial/plot_gauss.html#sphx-glr-modeling-gallery-spatial-plot-gauss-py
# https://docs.gammapy.org/0.19/api/gammapy.modeling.models.GaussianSpatialModel.html#gammapy.modeling.models.GaussianSpatialModel

# lon_0, lat_0 = Center position
# sigma = Length of the major semiaxis of the Gaussian, in angular units.
# e = Eccentricity of the Gaussian (0<𝑒<1).
# phi = Rotation angle 𝜙: of the major semiaxis. Increases counter-clockwise from the North direction.
# frame{“icrs”, “galactic”}

def set_GaussianSpatialModel1(parameters):
  paramvalues=get_values(parameters)
  get_attributes(parameters)  
  spatialmodel= GaussianSpatialModel(
    lon_0=paramvalues['RA']* u.deg, 
    lat_0=paramvalues['DEC']* u.deg,
    sigma=paramvalues['Sigma']* u.deg,
    e=0, phi=0* u.deg,
    frame="icrs"
  )
  return spatialmodel
###########################################
#   EllipticalGaussian  to  GaussianSpatialModel
########################################### 
# http://cta.irap.omp.eu/ctools/users/user_manual/models_spatial.html#ellipticalgaussian
# -> 
# Mspatial(θ,ϕ)=exp(−θ22r2eff),
# with
# reff=ab/√‾‾(asin(ϕ−ϕ0))^2+(bcos(ϕ−ϕ0))^2‾‾‾
# (I correct the description here. The one in the link should be wrong. )
# where
# RA is the Right Ascension (degrees)
# DEC is the Declination (degrees)
# PA is the position angle, counted counterclockwise from North (degrees)
# a = MinorRadius (degrees)
# b = MajorRadius (degrees)
# ϕ0 is the position angle of the ellipse, counted counterclockwise from North
# ϕ is the azimuth angle with respect to North.
def set_GaussianSpatialModel2(parameters):
  paramvalues=get_values(parameters)
  get_attributes(parameters)  
  spatialmodel= GaussianSpatialModel(
    lon_0=paramvalues['RA']* u.deg, 
    lat_0=paramvalues['DEC']* u.deg,
    e= np.sqrt(1 - (paramvalues['MinorRadius']/paramvalues['MajorRadius'])**2),
    phi=paramvalues['PA']* u.deg,
    frame="icrs"
  )
  return spatialmodel
#
###########################################
#   DiffuseMap  to  TemplateSpatialModel
########################################### 
# http://cta.irap.omp.eu/ctools/users/user_manual/models_spatial.html#diffusemap
# The DiffuseMap model describes an arbitrary intensity distribution in form of a sky map
# where
# Normalization is a normalization value
# NOTE: only one parameter
# -> 
#https://docs.gammapy.org/0.19/api/gammapy.modeling.models.TemplateSpatialModel.html#gammapy.modeling.models.TemplateSpatialModel
def set_TemplateSpatialModel1(parameters,filename):
  paramvalues=get_values([parameters])
  filepath=os.path.join(modelfilepath, filename)
  m = Map.read(filepath) 
  if paramvalues['Normalization']==1:
    if paramvalues['Normalization']==1:
      spatialmodel= TemplateSpatialModel(m, filename=filepath, normalize=True)
    else:
      print('Normalization is possible only to unity in gammapy. Spatial model generation is skipped.')
  else:
    spatialmodel= TemplateSpatialModel(m, filename=filepath, normalize=False)    

  return spatialmodel

#
###########################################
#   DiffuseMapCube  to  TemplateSpatialModel
########################################### 
# http://cta.irap.omp.eu/ctools/users/user_manual/models_spatial.html#diffusemapcube
#The DiffuseMapCube model describes an arbitrary energy-dependent intensity distribution in form of a map cube
# where
# Normalization is a normalization value
# -> 
# https://docs.gammapy.org/0.19/tutorials/api/models.html#Models-with-energy-dependent-morphology
#
#
def set_TemplateSpatialModel2(parameters,filename):
  paramvalues=get_values([parameters])
  filepath=os.path.join(modelfilepath, filename)
  m = Map.read(filepath) 
  if paramvalues['Normalization']==1:
    if paramvalues['Normalization']==1:
      spatialmodel= TemplateSpatialModel(m, filename=filepath, normalize=True)
    else:
      print('Normalization is possible only to unity in gammapy. Spatial model generation is skipped.')
  else:
    spatialmodel= TemplateSpatialModel(m, filename=filepath, normalize=False)    
  
  return spatialmodel



Registry
--------

ConstantSpatialModel           : ['ConstantSpatialModel', 'const'] 
TemplateSpatialModel           : ['TemplateSpatialModel', 'template'] 
DiskSpatialModel               : ['DiskSpatialModel', 'disk'] 
GaussianSpatialModel           : ['GaussianSpatialModel', 'gauss'] 
GeneralizedGaussianSpatialModel: ['GeneralizedGaussianSpatialModel', 'gauss-general'] 
PointSpatialModel              : ['PointSpatialModel', 'point'] 
ShellSpatialModel              : ['ShellSpatialModel', 'shell'] 
Shell2SpatialModel             : ['Shell2SpatialModel', 'shell2'] 



# Region selection

In [3]:

import numpy as np
from astropy.io import fits
import astropy.units as u

def get_values(parameters):
  paramvalues={}
  for parameter in parameters :    
    paramvalues[parameter['@name']]=float(parameter['@value'])*float(parameter['@scale'])
  return paramvalues

def get_galactic_edges_from_2dimfits(wcs, image) : 
  im_gal_lon_npix=image.shape[0]
  im_gal_lat_npix=image.shape[1]
  lon1,lat1 = wcs.all_pix2world( 0, 0, 0)
  lon2,lat2 = wcs.all_pix2world(im_gal_lon_npix,im_gal_lat_npix, 1)
  im_gal_lonmin, im_gal_lonmax = sorted([lon1,lon2])
  im_gal_latmin, im_gal_latmax = sorted([lat1,lat2])
  return im_gal_lonmin, im_gal_lonmax, im_gal_latmin, im_gal_latmax

def get_galactic_edges_from_3dimfits(wcs, image) : 
  #assume that wcs.array_shape=(energybins, xbins, ybins)
  # where xbins=ybins
  xbins=wcs.array_shape[1]
  ybins=wcs.array_shape[2]
  if (xbins!=ybins) : 
    print('!!!!!!!!!!!! exception happened!!!!!!!! xbins={} and ybins={} '.format(xbins,ybins))
  # x=np.arange(xbins)
  # y=np.arange(ybins)
  ncoarsebins=10
  x = np.linspace(0,xbins-1, ncoarsebins, dtype=int)
  y = np.linspace(0,ybins-1, ncoarsebins, dtype=int)
  from astropy.wcs.wcsapi import SlicedLowLevelWCS
  slices = [0, slice(0, xbins), slice(0, ybins)]  
  subwcs = SlicedLowLevelWCS(wcs, slices=slices) 
  import itertools
  pixelgrid=list(itertools.product(x, y))
  mapcoord_icrs=subwcs.pixel_to_world_values(pixelgrid[0],pixelgrid[1], 1)
  from astropy.coordinates import SkyCoord  # High-level coordinates
  import astropy.units as u
  c = SkyCoord(mapcoord_icrs[0], mapcoord_icrs[1], frame="icrs", unit="deg")  # 3 coords
  galactic=c.galactic
  im_gal_lonmin = galactic.l.deg.min()
  im_gal_lonmax = galactic.l.deg.max()
  im_gal_latmin = galactic.b.deg.min()
  im_gal_latmax = galactic.b.deg.max()  
  return im_gal_lonmin, im_gal_lonmax, im_gal_latmin, im_gal_latmax

from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
def see_if_in_the_region(ct_spatialinfo,l_min=60, l_max=80, b_min=-4, b_max=4 ):
  spatialtype=ct_spatialinfo["@type"]
  # print(spatialtype)
  if spatialtype=="DiffuseMap" or spatialtype=="DiffuseMapCube" :
    # print(ct_spatialinfo["@file"])
    modelfitsfilepath=os.path.join(modelfilepath ,ct_spatialinfo["@file"])
    hdul = fits.open(modelfitsfilepath)
    # print('========== %s ========'% (modelfitsfilepath))

    hdul_i=0
    image = hdul[hdul_i].data
    while (image is None) :
      hdul_i=hdul_i+1
      image = hdul[hdul_i].data
    wcs = WCS(header=hdul[0].header)
    if image.ndim>2 :
      im_gal_lonmin, im_gal_lonmax, im_gal_latmin, im_gal_latmax \
        =get_galactic_edges_from_3dimfits(wcs, image)
    else : 
      im_gal_lonmin, im_gal_lonmax, im_gal_latmin, im_gal_latmax \
        =get_galactic_edges_from_2dimfits(wcs, image)
      
    is_inside= ((im_gal_lonmax > l_min and im_gal_lonmin < l_max )
                 and 
               (im_gal_latmax > b_min and im_gal_latmin < b_max ))
    return is_inside
  else : 
    paramvalues=get_values(ct_spatialinfo["parameter"])
    sourcedirection=SkyCoord(ra=paramvalues['RA']*u.deg, dec=paramvalues['DEC']*u.deg, frame='icrs')
    # print(sourcedirection.galactic)
    is_inside= (sourcedirection.galactic.l.degree>l_min and
                sourcedirection.galactic.l.degree<l_max and 
                sourcedirection.galactic.b.degree<b_min and 
                sourcedirection.galactic.b.degree<b_max 
                )
    # print(is_inside) 
    return is_inside


In [7]:
import os
modelfilepath="/Users/kazuma/Workspace/CTA/20221012_NewSkymdlChk/12_Software to assemble Galactic models/gps-luigitibaldo/skymodel/output/"
modelxmlfilepath=os.path.join(modelfilepath ,"models_gps.xml")
outyamlfilename="outputs/spectral_spatial_models_cygnus.yaml"

########################################################
#   Spectral + spatial + save
########################################################
import xmltodict
import time
tic = time.perf_counter()
with open(modelxmlfilepath, encoding='utf-8') as fp:
  xml_data = fp.read()
  # xml -> dict
  dict_data = xmltodict.parse(xml_data)  
  dict_data_subset = dict_data["source_library"]["source"]
  
  firstEvtNo=0  
  EvtNo=firstEvtNo

  from gammapy.modeling.models import Models
  skymodels = Models()
  for data in dict_data_subset:
    gp_spectralmodel=None
    gp_spatialmodel= None
    gp_temporalmodel=None
    EvtNo=EvtNo+1
    # print(data)
    # print('================================= %d ====================================='% (EvtNo))
    # print('Source name: ' , data['@name'])
    # if data['@name']=="":
    #   continue
    # if EvtNo==106 or EvtNo==109 or EvtNo==110 or EvtNo==114: <-Composite spectral type
  #   # if EvtNo==1637:
  #   #   print('-----> skipped')
  #   #   continue
    # if ct_spectralinfo["@type"]=="Composite" :
    #   print("{}, {}".format(data['@name'],spatialtype) )
  #     continue


    # ======= Selecting Cygnus Region =============
    is_cygnus=see_if_in_the_region(data["spatialModel"],l_min=60, l_max=80, b_min=-4, b_max=4 )
    if not is_cygnus : 
      continue
    else :
      ct_spatialinfo=data["spatialModel"]
      spatialtype=ct_spatialinfo["@type"] 
      print("{}, {}".format(data['@name'],spatialtype) )
  
  

    ct_spectralinfo = data["spectrum"]    
    gp_spectralmodel=generate_spectralmodel(ct_spectralinfo)
    print(gp_spectralmodel)

    if "spatialModel" in data.keys():
      print('++++++++++ spatial info')      
      ct_spatialinfo = data["spatialModel"]
      ct_spatial_parameters = ct_spatialinfo["parameter"]      
      print('__________  paraeters in ctools format _______________________________')
      print("Spatial type: ", ct_spatialinfo["@type"], "with ", len(ct_spatial_parameters), "parameters")
      gp_spatialmodel=generate_spatialmodel(ct_spatialinfo)
      print(gp_spatialmodel)
    skymodel = SkyModel(
      name=data['@name'],
      spectral_model= gp_spectralmodel,
      spatial_model=gp_spatialmodel
      )
    skymodels.append(skymodel)
  skymodels.write(outyamlfilename, overwrite=True) 
  toc = time.perf_counter()
  print(f"processes {EvtNo-firstEvtNo:d} source models in {toc - tic:0.4f} seconds")      


Set MJD-END to 56871.689780 from DATE-END'. [astropy.wcs.wcs]


GammaCygni, DiffuseMap
@ generate_spectralmodel: This spectral model is  BrokenPowerLaw
BrokenPowerLawSpectralModel

  type      name     value         unit      ... max frozen is_norm link
-------- --------- ---------- -------------- ... --- ------ ------- ----
spectral    index1 2.0500e+00                ... nan  False   False     
spectral    index2 2.7900e+00                ... nan  False   False     
spectral amplitude 1.0895e-15 cm-2 MeV-1 s-1 ... nan  False    True     
spectral    ebreak 1.0000e+05            MeV ... nan  False   False     
++++++++++ spatial info
__________  paraeters in ctools format _______________________________
Spatial type:  DiffuseMap with  6 parameters
TemplateSpatialModel



the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Invalid parameter values: MJD-OBS and DATE-OBS are inconsistent'. [astropy.wcs.wcs]


3FHL J2053.8+2922, PointSource
@ generate_spectralmodel: This spectral model is  Multiplicative
@ generate_spectralmodel: This spectral model is  PowerLaw
first spectrum done
second spectrum done expct
CompoundSpectralModel
    Component 1 : PowerLawSpectralModel

  type      name     value         unit      ... max frozen is_norm link
-------- --------- ---------- -------------- ... --- ------ ------- ----
spectral     index 2.3950e+00                ... nan  False   False     
spectral amplitude 4.3891e-15 cm-2 MeV-1 s-1 ... nan  False    True     
spectral reference 1.0000e+00            TeV ... nan   True   False     
    Component 2 : ExpCutoffPowerLawNormSpectralModel

  type      name      value     unit ...    max     frozen is_norm link
-------- --------- ----------- ----- ... ---------- ------ ------- ----
spectral     index  0.0000e+00       ... -1.000e+01  False   False     
spectral      norm  1.0000e+00       ...        nan   True    True     
spectral reference  1.0000e+

Template file already exits, and overwrite is False


processes 2138 source models in 9.5585 seconds
